# MDL resistance

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
import pandas as pd
from vessel_manoeuvring_models.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,4)
from vessel_manoeuvring_models.symbols import *

In [ ]:
from scipy.optimize import least_squares

In [ ]:
data_TT_MDL = catalog.load('force_regression.data_TT_MDL')
R_m = data_TT_MDL['Rm [N]']
df_VCT = catalog.load('force_regression.data_scaled')

In [ ]:
def res(x, u):
    
    r = x*u**2
    
    return r

def error(x,y):
    
    u = y.index
    r = res(x, u)
    e = r - y
    return e

In [ ]:
result = least_squares(fun=error, x0=[0], kwargs={'y':R_m})

In [ ]:
df_resistance = df_VCT.groupby(by='test type').get_group('resistance').copy()
df_resistance['R'] = -df_resistance['fx']

In [ ]:
df = data_TT_MDL.copy()
u = R_m.index
df['R_pred'] = res(result.x, u)

fig,ax=plt.subplots()
df.plot(style='.-', ax=ax);
df_resistance.sort_values(by='u').plot(x='u',y='R', ax=ax)

In [ ]:
runs_meta_data = catalog.load('runs_meta_data')
runs_meta_data.dropna(how='any', axis=1, inplace=True)

In [ ]:
runs_meta_data.head()

In [ ]:
runs_meta_data.columns

In [ ]:
reference_speeds = runs_meta_data.groupby(by='test_type').get_group('reference speed')

In [ ]:
df_mean = pd.DataFrame()
df_std = pd.DataFrame()


for id, row in reference_speeds.iterrows():
    
    try:
        df_ = catalog.load(f'{id}.data_ek_smooth').iloc[400:-400]
    except:
        continue
    
    s_mean = df_.mean()
    s_mean.name = id
    df_mean = df_mean.append(s_mean)
    
    s_std = df_.std()
    s_std.name = id
    df_std = df_std.append(s_std)
    
    

In [ ]:
df_upper = df_mean.copy()
df_upper['thrust']+=3*df_std['thrust']
df_lower = df_mean.copy()
df_lower['thrust']-=3*df_std['thrust']
df_mean.sort_values(by='u', inplace=True)
df_upper.sort_values(by='u', inplace=True)
df_lower.sort_values(by='u', inplace=True)

In [ ]:
fig,ax=plt.subplots()

df_mean.plot(x='u', y='thrust', style='.-', ax=ax)
df_lower.plot(x='u', y='thrust', style='-', label='lower', ax=ax)
df_upper.plot(x='u', y='thrust', style='-', label='upper', ax=ax)
ax.legend();


In [ ]:
#tdf = 0.117
tdf = 0.117

R_factor =1.14
for df_ in [df_mean, df_lower, df_upper]:
    df_['R'] = (1-tdf)*df_['thrust']/R_factor

In [ ]:
df = data_TT_MDL.copy()
u = R_m.index
df['R_pred'] = res(result.x, u)

fig,ax=plt.subplots()
df.plot(style='.-', ax=ax);
df_resistance.sort_values(by='u').plot(x='u',y='R', ax=ax)

df_mean.plot(x='u', y='R', style='.-', ax=ax)

ax.fill_between(x=df_lower['u'], y1=df_lower['R'], y2=df_upper['R'], alpha=0.5, color='gray', label='95%')

#df_lower.plot(x='u', y='R', style='-', label='lower', ax=ax)
#df_upper.plot(x='u', y='R', style='-', label='upper', ax=ax)

ax.legend();
ax.set_xlim(0.6,1.1);
ax.set_ylim(3,10);

In [ ]:
1-0.117